resources:
https://serpapi.com/google-play-product-api


In [1]:
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all
import pandas as pd
import numpy as np

C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Ryan Jewik\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import string
def string_to_integer(input_str):
    # Remove punctuation from the string
    cleaned_str = input_str.translate(str.maketrans('', '', string.punctuation))
    # Convert the cleaned string to an integer
    try:
        return int(cleaned_str)
    except ValueError:
        raise ValueError("The input string does not contain a valid integer after removing punctuation.")

import re

def clean_filename(filename):
    # Define a regex pattern for characters not allowed in filenames
    invalid_chars = r'[\/:*?"<>|]'
    # Replace invalid characters with an underscore
    cleaned_filename = re.sub(invalid_chars, '_', filename)
    return cleaned_filename

reviews_all function returns all of reviews from app. If you want to set the count to infinity while using the reviews function, you can use the reviews_all function.

:bulb: Because of the Google Play Store limit (up to 200 reviews can be fetched at a time), http requests are generated as long as the number of app reviews is divided by 200. For example, targeting an app like Pokémon GO makes tens of thousands of http requests.

In [35]:
from google_play_scraper import search

#app search
result = search(
    "khan academy kids",
    lang="en",  # defaults to 'en'
    country="us",  # defaults to 'us'
    n_hits=30  # defaults to 30 (= Google's maximum)
)

In [36]:
result

[{'appId': 'org.khankids.android',
  'icon': 'https://play-lh.googleusercontent.com/aeB2AuucPeHi3Ph4eNDTG0AAJl5vPVWmqA4tI5soBRdWl9cR8EQ1GEzYtH7_rt00EqA',
  'screenshots': ['https://play-lh.googleusercontent.com/ElCQFvMOk629izLfxZnnrGjHQ6L15ZXYsWnJiAMBVXrnB8_m1SLk9QviyEwTvdWbZXU',
   'https://play-lh.googleusercontent.com/70FxMvwYw1HIzi0KsQa0VakwBXceziFF6xLAGAL8B5JzBryTggna5A799KejZjkuYjow',
   'https://play-lh.googleusercontent.com/RxzLictEc3VK8BgBvmWVkWrJah4POIBf7a3roB2bPjy2QBqzn0Os4NytmVcOPfnCsQ',
   'https://play-lh.googleusercontent.com/26KxsKvZQopCWnLyVRJr7_d8pB0dESVTfQ79tg81W7qNngVku8nu4wBzZk5mN0dG7ls',
   'https://play-lh.googleusercontent.com/9fXT6nCAzB36Z3wsVoAx4wWMideM9L8AHhuJ5HyCH8uJc0K04Xyu7ZXho1biuaJf5JCv',
   'https://play-lh.googleusercontent.com/N2n3xKahPDT8EDLquCUJ0JtUaxITMs7VoyWzdeNYPlQZGFgQaT_pBc_LYh18JqTDQA',
   'https://play-lh.googleusercontent.com/WDSk6fq_n-ECteLuDCJxiyOhll95UlX13D1rBhl3toEAl8e2aqcZgbaq1W8IX7OUBr39',
   'https://play-lh.googleusercontent.com/TmVG

khan academy kids:
 - org.pbskids.gamesapp
toddler games for 2+ year olds:
 - com.happytools.learning.kids.games
abc kids tracing & phonics:
 - com.kids.learn.reading.app

In [50]:
appIds = []
namelist = ["khan-academy-kids","toddler-games-for-2-year-olds", "abc-kids-tracing-phonics"]
appIds.append("org.khankids.android")
appIds.append("com.bimiboo.playandlearn")
appIds.append("com.kids.learn.reading.app")
applist = appIds

In [51]:
#we use this function if we want total reviews period
#applist = []
#namelist = []
for idx in range(0, 3):
    appid = appIds[idx]
    appcall = app(
        appid,
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
    )
    #there is a distinction between ratings and reviews, reviews have bodies of text while ratings only need the score
    print(f"{idx} ------------------")
    print(appcall['title'])
    print(appcall['score'])
    print(appcall['reviews'])
    print(appcall['appId'])
    
    #applist.append(result[idx]['appId'])
    #namelist.append(result[idx]['title'])
    
            

0 ------------------
Khan Academy Kids
4.646433
5834
org.khankids.android
1 ------------------
Toddler Games for 2+ year olds
4.448555
1398
com.bimiboo.playandlearn
2 ------------------
ABC Games: Phonics & Tracing
4.62234
1157
com.kids.learn.reading.app


In [54]:
namelist

['khan-academy-kids',
 'toddler-games-for-2-year-olds',
 'abc-kids-tracing-phonics']

In [55]:
applist

['org.khankids.android',
 'com.bimiboo.playandlearn',
 'com.kids.learn.reading.app']

In [56]:
#adds all the reviews to a dataframe
fileNameList = []
for idx in range(0, len(applist)):
    app_df = pd.DataFrame()
    result = reviews_all(
        applist[idx],
        sleep_milliseconds=0, # defaults to 0
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
        sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    )
    for review in result:
        developerResponse = review
        date = review['at']
        userName = review['userName']
        reviewText = review['content']
        score = review['score']
        #can't get title
        #can't get isEdited
        review_df = pd.DataFrame({'developerResponse': [developerResponse],  'date': [date], 'review': [reviewText], 
                                   'score': [score], 'userName': [userName],})
        fileName = "final/googleplayscraper-" + clean_filename(namelist[idx])
        app_df = pd.concat([app_df, review_df], ignore_index = True)
    fileNameList.append(fileName)
    app_df.to_csv(fileName + ".csv")
    #rand200 = app_df.sample(n=100, random_state=101, replace = True)
    #rand200.to_csv("rand200/googleplayscraper-" + clean_filename(namelist[idx]), index = False)

 - date
 - json response (developerResponse)
 - review text (review)
 - rating
 - isEdited
 - userName
 - title

In [57]:
app_df

,developerResponse,date,review,score,userName
0,{'reviewId': '24acc1c4-821d-443f-9f13-9b91047c...,2024-03-05 05:27:14,"My 3 year old learns so much with this app, an...",3,A Google user
1,{'reviewId': '4d567b86-d141-4c59-b0d9-ec2a1c5d...,2020-07-05 08:34:49,This is a fantastic app for kids. My 2 year ol...,1,A Google user
2,{'reviewId': 'dedb4a9b-4af1-4fba-9d24-dcbd27da...,2022-03-11 17:40:42,+ 5 stars: Great game overall! - 1 star: Loud ...,3,A Google user
3,{'reviewId': 'f7fdedd8-a16f-4b86-81fe-86f0921c...,2024-09-17 03:53:29,"""Exceptional Learning Tool!"" ABC Games: Phonic...",5,A Google user
4,{'reviewId': 'd1b48173-204c-4786-89dc-541bd5d0...,2024-07-28 06:34:33,Very happy to use the application and it's ver...,5,A Google user
...,...,...,...,...,...
8552,{'reviewId': 'aeae5c66-4107-4937-b90a-51281283...,2021-08-23 09:55:01,👍👍👍👍👍,5,A Google user
8553,{'reviewId': 'f3a2aeb2-c1e2-469e-873f-6575aa93...,2021-12-06 22:45:11,🤙🤙🤙,5,A Google user
8554,{'reviewId': '3dfba630-9748-41bb-87ab-4ac8955e...,2020-09-13 19:32:36,😄😄😄😄,5,A Google user
8555,{'reviewId': '711880ed-fdec-44ae-9366-d1e5f182...,2021-08-12 08:35:16,😀😀,5,A Google user


In [34]:
fileNameList

['data/googleplayscraper-Lingokids - Play and Learn',
 'data/googleplayscraper-ABCmouse – Kids Learning Games',
 'data/googleplayscraper-ABC Kids - Tracing & Phonics',
 'data/googleplayscraper-Starfall',
 'data/googleplayscraper-Spelling & Phonics_ Kids Games']

In [24]:
#apps found via total reviews read by google play scraper
for idx in range(0, 5):
    df = pd.read_csv(f"{fileNameList[idx]}.csv")
    print(namelist[idx])
    print(df.shape)

ABCmouse – Kids Learning Games
(28410, 6)
ABC Kids - Tracing & Phonics
(18029, 6)
Starfall
(6259, 6)
Epic: Kids' Books & Reading
(32702, 6)
Khan Academy Kids
(12640, 6)
